# Scam Detector: Decision Tree

    This Jupyter Notebook will be used to run a Decision Tree Algorithm to predict if a given email is a scam or a ham(a normal email).

## Import Packages

In [100]:
#import the packages we need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Retrieve Data

In [119]:
test_col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
test_path = "IRIS.csv"
test = pd.read_csv(path)
test["species"] = pd.factorize(test['species'])[0]
test.head(100)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1
96,5.7,2.9,4.2,1.3,1
97,6.2,2.9,4.3,1.3,1
98,5.1,2.5,3.0,1.1,1


In [128]:
col_names = ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls']
path = "./data/CEAS_08.csv"
data = pd.read_csv(path)
col_names[-1], col_names[-2] = col_names[-2], col_names[-1]
data = data[col_names]
data.head(100)

,sender,receiver,date,subject,body,urls,label
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,1,0
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1
...,...,...,...,...,...,...,...
95,Brett Cannon <liivp@python.org>,ppcwedbyff@gmail.com,"Tue, 05 Aug 2008 16:35:11 -0700",Re: [Python-Dev] Python-Dev Summary Draft (Apr...,"On 4/24/07, Calvin Spealman wrote:\n> I have ...",1,0
96,Calvin Spealman <ppcwedbyff@gmail.com>,Talin <ovijn@acm.org>,"Tue, 05 Aug 2008 19:35:04 -0400",Re: [Python-Dev] Python-Dev Summary Draft (Apr...,"On 4/24/07, Talin wrote:\n> Calvin Spealman w...",1,0
97,Perl Jobs <xycn-vtnhz@perl.org>,necc@perl.org,"Tue, 05 Aug 2008 09:10:33 -0800",[Perl Jobs] Top NYC LAMP shop / B2B mod_perl /...,Online URL for this job: http://jobs.perl.org/...,1,0
98,Nancy Graziano <qgzon.djsmosok@gmail.com>,bspsggvwcfmo-bvbmb@lists.sourceforge.net,"Tue, 05 Aug 2008 18:35:55 -0500",[SM-USERS] SM 1.4.13 Configuration Question: M...,"Good Evening,\n\nI have tried setting the Mail...",0,0


## Decision Class

In [129]:
class Decision:
    """ A decision is used to ask the question at a decision node to split the data.
    This class records column number and values and matches the stored feature value to a give feature value
    """
    
    def __init__(self, feature_index, threshold):
        self.feature_index = feature_index
        self.threshold = threshold
        
    def ask(self, input):
        # Compares input feature value to stored value
        feature_val = input[self.feature_index]
        if isinstance(feature_val, (int, float)):
            return feature_val >= self.threshold
        else:
            return feature_val == self.threshold
        

## Helper Functions for Splitting

In [130]:
def divide_df(rows, decision):
    # Partitions a data frame
    # Check if each row matches decision, divide into true and false
    
    left, right = [], []
    for row in rows:
        left.append(row) if decision.ask(row) else right.append(row)
    return left, right

In [131]:
def label_count(rows):
    # Counts the number of each classification in data frame
    label_counts = {}
    for row in rows:
        label = row[-1]
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1
    return label_counts

In [132]:
def gini_impurity(rows):
    #Calculates Gini Impurity for a data frame of rows.
    
    label_counts = label_count(rows)
    gini = 1.0
    for label in label_counts:
        label_prob = label_counts[label]/float(len(rows))
        gini -= label_prob**2
    return gini

In [133]:
def info_gain(left, right, curr_gini):
    #Information gain: Gini of the root node subtracted by the impurty of the two children nodes.
    prob = float(len(left) / (len(left) + len(right)))
    return curr_gini - prob * gini_impurity(left) - (1 - prob) * gini_impurity(right)
                 

In [134]:
def info_gain_split(rows):
    #Find best decision to make based on informaiton gain
    highest_gain = 0
    optimal_decision = None
    curr_gini = gini_impurity(rows)
    feature_count = len(rows[0]) -1
    
    for feature_index in range(0, feature_count):
        
        unique = set(row[feature_index] for row in rows) #unique column values
        
        for candidate in unique:
            decision = Decision(feature_index, candidate)
            
            left, right = divide_df(rows, decision)
            
            #continue if no split
            if(len(left) == 0 or len(right) == 0):
                continue
            
            #information gain from this split
            gain = info_gain(left, right, curr_gini)
            
            if gain > highest_gain:
                highest_gain, optimal_decision = gain, decision
                
    return highest_gain, optimal_decision

## Build Tree and Node Classes

In [135]:
class LeafNode:
    # A leaf Node holdes classified data.
    # Holds a dictionary with class counts in the leaf.
    
    def __init__(self,rows):
        self.pred = label_count(rows)

In [136]:
class DecisionNode:
    # A Decision Node asks a Decision to be made.
    # Holds reference to a Decision, and two child nodes.
    
    def __init__(self, decision, left, right):
        self.decision = decision
        self.left = left
        self.right = right

In [137]:
def build_tree(rows):
    # Recursively Builds tree.
    
    highest_gain, optimal_decision = info_gain_split(rows)
    
    #Base case no further gain
    if highest_gain == 0:
        return LeafNode(rows)
    
    #Found Partition
    left, right = divide_df(rows, optimal_decision)
    
    #Recurse Left Subtree
    left_subtree = build_tree(left)
    
    #Recurse Right Subtree
    right_subtree = build_tree(right)
    
    #Return Decision Node
    return DecisionNode(optimal_decision, left_subtree, right_subtree)

In [139]:
df_list = data[0:100].values.tolist()
my_tree = build_tree(df_list)

In [140]:
def predict(row, curr_node):
    #Base Case: Curr node is a leaf
    if isinstance(curr_node, LeafNode):
        return curr_node.pred
    
    #Recurse the left or right subtree
    if curr_node.decision.ask(row):
        return predict(row, curr_node.left)
    else:
        return predict(row, curr_node.right)

In [143]:
predict(df_list[3], my_tree)

{0: 1}